In [1]:
import numpy as np
from cogent3 import get_app, open_data_store
from cogent3.app.composable import define_app, NotCompleted
from cogent3.maths.measure import jsd
import scipy.stats as stats
from scipy.spatial.distance import correlation
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from clock_project.simulation.wts import taxonomic_triple_simulation, three_seq_simulation_star_shape, generate_ancestor


In [3]:
import json
with open ('../ensembl_genome_analysis/matrix_full.json', 'r') as infile:
    matrices_dict = json.load(infile)

In [4]:
matrix_collection = []
for gene, matrices_list in matrices_dict.items():
    matrix_collection.extend(matrices_list)

In [6]:
Q_ingroup1, Q_ingroup2, Q_outgroup = matrices_dict['ENSG00000020129'][0].values()

In [7]:
Q_ingroup1_, Q_ingroup2_, Q_outgroup_ = matrices_dict['ENSG00000048707'][0].values()

In [8]:
p0 = [0.25, 0.25, 0.25, 0.25]

In [10]:
path1 = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_350_threshold/ENSG00000007933/model_fitting_result/0.json'
load_json_app = get_app("load_json")
res = load_json_app(path1)

In [13]:

def get_seed_ancester_seq(pi_low, pi_high, len_short, len_long):
    seq_ls = generate_ancestor(len_short, pi_low)
    seq_ll = generate_ancestor(len_long, pi_low)
    seq_hs = generate_ancestor(len_short, pi_high)
    seq_hl = generate_ancestor(len_long, pi_high)
    seqs = {'low_short': seq_ls, 'high_short': seq_hs, 'low_long': seq_ll, 'high_long': seq_hl}
    return seqs


In [14]:
from cogent3 import open_data_store
def simulate_taxanomic_triples(ancester_seq, Q_collection, pi, seed, t, path_to_dir):
    length = len(ancester_seq)
    for matrix_dict in Q_collection:
        Q_ingroup1, Q_ingroup2, Q_outgroup = matrices_dict.values()
        aln, ens = taxanomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup), 0, t, length, 1, 123, ancester_seq)
        path_to_dir = '/Users/gulugulu/Desktop/honours/data_local/bootstrapping_test_non'
        out_dstore = open_data_store(path_to_dir, mode="w", suffix="json")
        write_json_app = get_app("write_json", data_store=out_dstore)
        write_json_app(aln, identifier = Q_collection.index(matrix_dict))






In [15]:
ances_seq = generate_ancestor(300, p0)

In [16]:
path_to_dir = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln'
out_dstore = open_data_store(path_to_dir, mode="w", suffix="json")
write_json_app = get_app("write_json", data_store=out_dstore)
Q_ingroup1, Q_ingroup2, Q_outgroup = matrix_collection[0].values()
aln, ens = taxonomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup), 0, 1, 1000, 1, 123, ances_seq)
# Open a data store for each simulation
write_json_app(aln, identifier=f'{matrix_collection.index(matrix_collection[0])}.json')

DataMember(data_store=/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln, unique_id=0.json)

In [17]:
import itertools

def generate_combinations(time_list, length_list, pi_list):
    # Create all combinations of time, length, and pi
    combinations = itertools.product(time_list, length_list, pi_list)
    return list(combinations)

# Example lists
time_list = [0, 1]
length_list = [100, 1000]
pi_list = ['pi1', 'pi2']

# Generate and print all combinations
combinations = generate_combinations(time_list, length_list, pi_list)


In [49]:
aln1, ens1 = taxonomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup), 0, 1, 300, 1, 123, ances_seq)

In [39]:
aln2, ens2= taxonomic_triple_simulation(p0, np.array(Q_ingroup1), np.array(Q_ingroup2), np.array(Q_outgroup_), 0, 1, 300, 1, 123, ances_seq)

Model fitting for all the simulated alignment

In [66]:
from cogent3.app.typing import AlignedSeqsType, SerialisableType, IdentifierType

@define_app
def model_fitting(aln: AlignedSeqsType, opt_args = None) -> SerialisableType:
    tree="('outgroup_edge3',('ingroup_edge1','ingroup_edge2')'internal_node');"
    GN_model = get_app("model", "GN", 
                    tree=tree,
                    opt_args=opt_args,
                    # unique_trees=True,
                    # lf_args=dict(discrete_edges=[['outgroup_edge3']]),
                    time_het="max",
                    optimise_motif_probs=True)
    res = GN_model(aln)
    return res

model_fitting_app = model_fitting()

@define_app
def customised_load_json(DataMember: IdentifierType) -> AlignedSeqsType:
    unique_id = DataMember.unique_id
    aln = load_json_app(DataMember)
    aln.source = unique_id
    return aln

def get_id(result):
    return result.alignment.source


load_json_app_customised = customised_load_json()
load_json_app = get_app("load_json")

outdir = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_model_fitting/aln_high_5000_1'
out_dstore = open_data_store(outdir, mode="w", suffix="json")
write_json_app = get_app("write_json", data_store=out_dstore, id_from_source = get_id)

simulated_aln_high_5000_2 = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln/aln_high_5000_2'
input_data_store_simulated_aln = open_data_store(simulated_aln_high_5000_2, suffix= 'json')



In [50]:
a = model_fitting_app(aln1)

In [63]:
a.lf.get_motif_probs_by_node()

root,0.1910,0.2553,0.2602,0.2935
outgroup_edge3,0.1933,0.2533,0.2666,0.2868
internal_node,0.2246,0.2263,0.2562,0.2929
ingroup_edge1,0.2233,0.2367,0.2500,0.2900
ingroup_edge2,0.1896,0.2770,0.2266,0.3068


In [55]:
ens1

{'internal_edge': 0.0,
 'ingroup_edge1': 0.11333333333333333,
 'ingroup_edge2': 0.23333333333333334,
 'outgroup_edge': 0.03333333333333333}

In [56]:
aln1

,0
ingroup_edge1,CTCCGCGTGCAGCCCCACGCATTGGACAGCAATATGCGGCTGATCCGGTAACCAGGACGC
ingroup_edge2,TAT..T..........G.............C..G..T.....C.TT....C.T......T
outgroup_edge3,TAT.............C......A........CG..T......C.T..............


In [51]:
res = a.lf

In [52]:
res.get_motif_probs_by_node()

root,0.1910,0.2553,0.2602,0.2935
outgroup_edge3,0.1933,0.2533,0.2666,0.2868
internal_node,0.2246,0.2263,0.2562,0.2929
ingroup_edge1,0.2233,0.2367,0.2500,0.2900
ingroup_edge2,0.1896,0.2770,0.2266,0.3068


In [54]:
res.get_lengths_as_ens()

{'outgroup_edge3': 0.011702916393696977,
 'ingroup_edge1': 0.09029985191773744,
 'ingroup_edge2': 0.24013567622653245,
 'internal_node': 0.043613544647546035}

In [60]:
aln.names

['ingroup_edge1', 'ingroup_edge2', 'outgroup_edge3']

In [37]:
model_fitting_path(input_data_store_simulated_aln[0])

DataMember(data_store=/Users/gulugulu/Desktop/honours/data_local/simulated_triple_model_fitting/aln_high_3000_2, unique_id=729.json)

In [79]:
from clock_project.simulation.wts import calculate_non_stationarity

def get_triads_info(path):
    aln = load_json_app_customised(path)
    model_fitting_result = model_fitting_app(aln)
    seq_names = aln.names
    matrices = {n: model_fitting_result.lf.get_rate_matrix_for_edge(n, calibrated=True) for n in seq_names}
    sub_ens_tree = model_fitting_result.lf.get_ens_tree()
    lca_node = sub_ens_tree.lowest_common_ancestor(['ingroup_edge1', 'ingroup_edge2']).get_node_names()[0]
    ingroup_seqs = aln.take_seqs(['ingroup_edge1', 'ingroup_edge2'])
    nuc_freqs = ingroup_seqs.probs_per_seq()
    nuc_freqs1 = nuc_freqs['ingroup_edge1']
    nuc_freqs2 = nuc_freqs['ingroup_edge2']
    ingroup_jsd = jsd(nuc_freqs1, nuc_freqs2)
    distribution_internal_root = list(model_fitting_result.lf.get_motif_probs_by_node()[lca_node])
    jsd1 = jsd(nuc_freqs1, distribution_internal_root)
    jsd2 = jsd(nuc_freqs2, distribution_internal_root)
    distribution_root = list(model_fitting_result.lf.get_motif_probs_by_node()['root'])
    ens_dict = {n: sub_ens_tree.to_rich_dict()['edge_attributes'][n]['length'] for n in seq_names}
    ens_difference = abs(ens_dict['ingroup_edge1']-ens_dict['ingroup_edge2'])
    jsd_dict = {'Ingroup_JSD': ingroup_jsd, 'JSD_difference': {'ingroup_edge1': jsd1, 'ingroup_edge2': jsd2}}
    nabla_dict = {}
    for species in ['ingroup_edge1', 'ingroup_edge2']:
        nabla_dict[species] = calculate_non_stationarity(distribution_internal_root, matrices[species], ens_dict[species])
    nabla_dict['outgroup_edge3'] = calculate_non_stationarity(distribution_root, matrices['outgroup_edge3'], ens_dict['outgroup_edge3'])
    shortest_ens = min(ens_dict.values())  
    matrices_list = {n: matrices[n].to_array().tolist() for n in seq_names}
    distributions = {'internal_node': distribution_internal_root, 'root': distribution_root, 'ingroup1': list(nuc_freqs1), 'ingroup2': list(nuc_freqs2)}
    triad_data = {'triads_species_names': seq_names, 'triads_info_small_tree': 
        {'ens_difference': ens_difference,
        'shortest_ingroup_ens': shortest_ens,
        'ens': ens_dict,
        'jsd_dict': jsd_dict,
        'nuc_freqs_dict': distributions,
        'nabla_values': nabla_dict,
        'matrices': matrices_list}}
    return triad_data, model_fitting_result

In [80]:
info, res = get_triads_info(input_data_store_simulated_aln[0]) 

In [84]:
res.alignment.source.split('.')[0]

'729'

In [87]:
import glob
import os
path = '/Users/gulugulu/Desktop/honours/data_local/simulated_triple_aln/aln_high_500_2'
triads_alignment_paths = glob.glob(os.path.join(path, "*.json"))

In [90]:
load_json_app_customised(triads_alignment_paths[0])

NotCompleted(type=ERROR, origin=customised_load_json, source="729.json", message="Traceback (most recent call last):
  File "/Users/gulugulu/opt/anaconda3/envs/c312/lib/python3.12/site-packages/cogent3/app/composable.py", line 384, in _call
    result = self.main(val, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gulugulu/opt/anaconda3/envs/c312/lib/python3.12/site-packages/cogent3/app/composable.py", line 444, in _main
    return self._user_func(**bound.arguments)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/d8/pdrt51hx2jb17vf6k28_x6mh0000gn/T/ipykernel_4101/3093290215.py", line 20, in customised_load_json
    unique_id = DataMember.unique_id
                ^^^^^^^^^^^^^^^^^^^^
AttributeError: 'str' object has no attribute 'unique_id'
")

In [ ]:
load_json_app()